# Day 1

## Imports and data loading

In [1]:
from utils import get_input, load_data

day = 8


In [5]:
get_input(day)


Data saved


In [2]:
data = load_data(day, list_type="line", number=False)
test_data = [
    "be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe",
    "edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc",
    "fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg",
    "fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb",
    "aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea",
    "fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb",
    "dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe",
    "bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef",
    "egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb",
    "gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce",
]
test_answer_1 = 26
test_answer_2 = 61229


## Prep

In [3]:
def split_data(inputs) -> tuple:
    divided = list(zip(*[line.partition(" | ") for line in inputs]))
    return divided[0], divided[2]


def count_digits(after) -> list:
    lengths = [[len(c.strip()) for c in codes.split()] for codes in after]
    return lengths


def count_uniques(counts) -> int:
    total = 0
    for line in counts:
        for val in line:
            if val in [2, 4, 3, 7]:
                total += 1
    return total


def part_one_answer(data) -> int:
    counts = count_digits(data)
    return count_uniques(counts)


## Part one

In [8]:
divided_test = split_data(test_data)
divided = split_data(data)

assert part_one_answer(divided_test[1]) == test_answer_1
part_one_answer(divided[1])


512

## Part two

In [14]:
# a
#b c
# d
#e f
# g

# Use differences between eg 4 and 1 to identify individual letters

# give each of a to g a set of possible translations, then gradually reduce them?
LETTERS = {"a", "b", "c", "d", "e", "f", "g"}
DISPLAYS = {
    ("a", "b", "c", "e", "f", "g"): 0,
    ("c", "f"): 1,
    ("a", "c", "d", "e", "g"): 2,
    ("a", "c", "d", "f", "g"): 3,
    ("b", "c", "d", "f"): 4,
    ("a", "b", "d", "f", "g"): 5,
    ("a", "b", "d", "e", "f", "g"): 6,
    ("a", "c", "f"): 7,
    ("a", "b", "c", "d", "e", "f", "g"): 8,
    ("a", "b", "c", "d", "f", "g"): 9,
}
INVERTED_DISPLAYS = {val: key for key, val in DISPLAYS.items()}

class LetterBoard():
    def __init__(self, line):
        divided = line.partition(" | ")
        self.codes = [set(i.strip()) for i in divided[0].split()]
        self.output = [set(d) for d in divided[2].split()]
        self.possible_letters = {letter: LETTERS.copy() for letter in LETTERS}

    def limit_letters(self, letters_used: set, intended_letters: str):
        for letter in self.possible_letters:
            if letter in letters_used:
                self.possible_letters[letter] &= set(intended_letters)
            else:
                self.possible_letters[letter] -= set(intended_letters)
            
            if len(actual_letter := self.possible_letters[letter]) == 1:
                self.translations[next(iter(actual_letter))] = letter

    def untangle(self):
        """Narrow down the possible letters represented by each of the used letters."""
        for code in self.codes:
            if len(code) == 2:
                self.limit_letters(code, "cf")
            elif len(code) == 3:
                self.limit_letters(code, "acf")
            elif len(code) == 4:
                self.limit_letters(code, "bcdf")

        for code in self.codes:
            if len(code) == 6:
                missing_code = next(iter(LETTERS - code))
                actual_missing = self.possible_letters[missing_code]
                actual_missing -= {"a", "b", "f", "g"}
                if len(actual_missing) == 1:
                    self.limit_letters(missing_code, actual_missing)

        for code in self.codes:
            if len(code) == 5:
                missing_codes = LETTERS - code
                actual_missing = set()
                for m in missing_codes:
                    actual_missing.update(self.possible_letters[m])
                actual_missing -= {"a", "d", "g"}
                if len(actual_missing) == 2:
                    self.limit_letters(missing_codes, actual_missing)

    def identify_numbers(self):
        """Ludicrous mess that repeats untangle, but I can't be bothered to tidy it."""
        digits = []
        for code in self.output:
            if len(code) == 2:
                number = 1
            elif len(code) == 3:
                number = 7
            elif len(code) == 4:
                number = 4
            elif len(code) == 5:
                missing = LETTERS - code
                possible_missing = set()
                possible = set()
                for m in missing:
                    possible_missing.update(self.possible_letters[m])
                for c in code:
                    possible.update(self.possible_letters[c])
                possible_missing -= {"a", "d", "g"}
                if possible_missing == {"b", "f"}:
                    number = 2
                elif possible_missing == {"b", "e"}:
                    number = 3
                elif possible_missing == {"c", "e"}:
                    number = 5
                else:
                    raise ValueError(f"Can't work out {code}")

            elif len(code) == 6:
                missing = list(LETTERS - code)[0]
                possible = self.possible_letters[missing]
                possible -= {"a", "b", "f", "g"}
                if len(possible) == 1:
                    if possible == {"d"}:
                        number = 0
                    elif possible == {"c"}:
                        number = 6
                    elif possible == {"e"}:
                        number = 9
                    else:
                        raise ValueError(f"Can't convert missing {possible} into a number.")
                else:
                    raise ValueError(f"Can't work out {code}.")
            elif len(code) == 7:
                number = 8
            else:
                raise ValueError(f"Length of {len(code)} can't be a valid number.")

            digits.append(str(number))
        
        return int("".join(digits))
        

    def convert_to_number(self):
        """Check all the letters have only one possible translation."""
        self.untangle()
        return self.identify_numbers()

def decode_and_add(lines):
    total = 0
    for line in lines:
        board = LetterBoard(line)
        result = board.convert_to_number()
        total += result
    return total

In [15]:
for test in test_data:
    board = LetterBoard(test)
    board.untangle()
    print(board.convert_to_number())
assert decode_and_add(test_data) == test_answer_2


8394
9781
1197
9361
4873
8418
4548
1625
8717
4315


In [16]:
decode_and_add(data)


1091165